#### This lecture notes have been copied and edited from 
#### http://www.langmead-lab.org/teaching-materials/ 

#### Hamming edit distance

Hamming edit distance is a basic score measuring how strings of equal length are different from each other. The difference count is basically the distance between two strings. 

The measure is limited to only strings of equal lengths.

In [ ]:
def hammingDistance(x, y):
    """
    Return Hamming distance between x and y 
    """
    if not len(x) == len(y): # check if the lengths are not equal
        return None # distance not defined
    nmm = 0
    for i in range(0, len(x)):
        if x[i] != y[i]:
            nmm += 1
    return nmm

print(hammingDistance('GCGTATGCACGC', 'GCTATGCCACGC'))

print(hammingDistance('GCGTATGCACGC', 'GCTATGCCACG'))


Sometimes it is useful to obtain all occurrences of $p$ within a longer sequence $p$ such that the Hamming distance is not longer than $m$.

In [ ]:
def naive_approx_hamming(p, t, m):
    # keep track of (position, distance) pairs
    occurrences = []
    # start from each (viable) position in t
    for i in range(0, len(t) - len(p) + 1):
        # calculate hamming distance (see above)
        nmm = 0
        for j in range(0, len(p)):
            if t[i+j] != p[j]:
                nmm += 1
                # stop prematurately if maximum distance is exceeded
                if nmm > m:
                    break
        # record matches as (position, distance) pairs
        if nmm <= m:
            occurrences.append((i,nmm))
    return occurrences

p = 'GCGA'
t = 'GCTATGCCACGC'
for pos,distance in naive_approx_hamming(p, t, 2):
    print(distance)
    print(p)
    print(t[pos:pos+len(p)])

So we can modify the Hamming edit distance to allow two strings/sequences of unequal length for calculation. By aligning the sequence to be compared from the first position, and different lengths are then added to distance. Missing the items to compared from shorter sequence is considered mismatch. 

In [ ]:
def boundEditDistance(x, y):
    """
    Return loose lower and upper bounds on the edit distance
    between x and y in O(|x| + |y|) time.
    """
    if x == y: 
        return 0, 0
    if len(x) == 0: 
        return len(y), len(y)
    if len(y) == 0: 
        return len(x), len(x)
    diff = abs(len(x) - len(y)) 
    lower = diff # the difference is at least the difference of the lengths of sequences 
    if lower == 0 and x != y: # if the sequences are different but the same length then the mininum difference is 1
        lower = 1
    minlen = min(len(x), len(y))
    upper = hammingDistance(x[:minlen], y[:minlen]) + diff # check with hamming edit distance only with the common lengths
    # the upper bound may become smaller by assigning the gaps differently (now they are all at the end, basically)
    # finding the optimal positions of gaps will result in the exact edit distance
    return lower, upper

boundEditDistance('GCGTATGCACGC', 'GCTATGCCACGC')


#### Edit distance with dynamic programming

Better algorithm for calculating string edit distance is done using dynamic programming. The algorithm allows three operations to be applied on sequences, insertion, substitution and deletion, thus sequences in comparison are not necessarily of the same lengths.  

In [ ]:
from numpy import zeros

def edDistDp(x, y):
    """ 
    Calculate edit distance between sequences x and y using
    matrix dynamic programming.  Return distance. 
    """
    D = zeros((len(x)+1, len(y)+1), dtype=int) # create zeros matrix of the length x and length y 
    
    D[0, 1:] = range(1, len(y)+1)
    D[1:, 0] = range(1, len(x)+1)
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            delt = 1 if x[i-1] != y[j-1] else 0
            D[i, j] = min(D[i-1, j-1]+delt, D[i-1, j]+1, D[i, j-1]+1)
    print('distance between sequence x and y =', D[len(x), len(y)])
    return D
    
print('with equal lenght of sequences')
D = edDistDp('GCGTATGCACGC', 'GCTATGCCACGC')
print(D)
print()
print('with different lengths of sequences')
D = edDistDp('GCGTATGCACGC', 'GCTATGCCACG')
print(D)

#### Global alignment

Then we can perform global alignment for two DNA sequences using dynamic programming. The three opeations mimick the evolutionary process including insertion, deletion and substition of a single or more nucleotide. The scoring function is arbitrarily defined here, but you can specify using tools like BLAST. There is no exact scoring matrix defined for each organism or DNA sequences, only recommendation for larger group of organisms. Selecting the right scoring matrix remains challenging.

In [ ]:
from numpy import zeros

def exampleCost(xc, yc):
    """ 
    Cost function assigning 
    0 to match, 
    2 to transition, 
    4 to transversion, and 
    8 to a gap 
    """
    if xc == yc: 
        return 0 # match
    if xc == '-' or yc == '-': 
        return 8 # gap
    minc, maxc = min(xc, yc), max(xc, yc)
    if minc == 'A' and maxc == 'G': 
        return 2 # transition
    elif minc == 'C' and maxc == 'T': 
        return 2 # transition
    return 4 # transversion

def globalAlignment(x, y, s):
    """ 
    Calculate global alignment value of sequences x and y 
    using dynamic programming.  
    Return global alignment value. 
    """
    D = zeros((len(x)+1, len(y)+1), dtype=int) # setting up zero matrix to whole values of alignment
    for j in range(1, len(y)+1): # assign gap score to every column of first row, meaning no x sequence, creating pure gap
        D[0, j] = D[0, j-1] + s('-', y[j-1])
    for i in range(1, len(x)+1): # assign gap score to the first column meaning no y sequence, creating pure gap
        D[i, 0] = D[i-1, 0] + s(x[i-1], '-')
    for i in range(1, len(x)+1): # dynamic programming part
        for j in range(1, len(y)+1): 
            # take minimum of these three scores from operation on sequences
            D[i, j] = min(D[i-1, j-1] + s(x[i-1], y[j-1]), # diagonal -- check whether it is match/transition/transversion
                          D[i-1, j  ] + s(x[i-1], '-'),    # vertical -- gap on y sequence
                          D[i  , j-1] + s('-',    y[j-1])) # horizontal -gap on x sequence
    return D, D[len(x), len(y)]

D, globalAlignmentValue = globalAlignment('TACGTCAGC', 'TATGTCATGC', exampleCost)
print(D)
print('alignment score = ', globalAlignmentValue)

#### Local alignment

We define a simple cost function exampleCost and a function smithWaterman that, given strings x and y and a cost function, fills in and returns the corresponding local alignment matrix.


In [ ]:
import numpy

def exampleCost(xc, yc):
    ''' Cost function: 2 to match, -6 to gap, -4 to mismatch '''
    if xc == yc: return 2 # match
    if xc == '-' or yc == '-': return -6 # gap
    return -4

def smithWaterman(x, y, s):
    ''' Calculate local alignment values of sequences x and y using
        dynamic programming.  Return maximal local alignment value. '''
    V = numpy.zeros((len(x)+1, len(y)+1), dtype=int)
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            V[i, j] = max(V[i-1, j-1] + s(x[i-1], y[j-1]), # diagonal
                          V[i-1, j  ] + s(x[i-1], '-'),    # vertical
                          V[i  , j-1] + s('-',    y[j-1]), # horizontal
                          0)                               # empty
    argmax = numpy.where(V == V.max())
    return V, int(V[argmax])

x, y = 'GGTATGCTGGCGCTA', 'TATATGCGGCGTTT'
V, best = smithWaterman(x, y, exampleCost)
print(V)
print("Best score=%d, in cell %s" % (best, numpy.unravel_index(numpy.argmax(V), V.shape)))
